In [24]:
import urllib, sys
from bs4 import BeautifulSoup
import re
from collections import Counter
import time
import csv
import pandas as pd
from urllib.error import HTTPError


In [25]:
tickerList = pd.read_excel('../CallTranscript_DownloadTemplate.xlsx')

In [29]:
wordlist = ['transcript']
wordlist.extend([str(tickerList['Year'][0]), str(tickerList['Quarter'][0])])
wordlist

['transcript', '2019', 'q1']

In [ ]:
symbol_list = list(tickerList['Ticker'])
# symbol_list=['GOOG','GOOGL','GE','ETN', 'PEP', 'KHC', 'FB', 'MCHP', 'SLAB', 'JPM', 'STT']
# df=pd.DataFrame(columns=['ticker','word','count'])
transcriptText = ''

for symbol in symbol_list:
    site='http://seekingalpha.com/symbol/'+symbol+'/earnings/transcripts'
    print(site)

    hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
       'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
       'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
       'Accept-Encoding': 'none',
       'Accept-Language': 'en-US,en;q=0.8',
       'Connection': 'keep-alive'}
    
    
    try:
        pageReq=urllib.request.Request(site,headers=hdr) 
        page=urllib.request.urlopen(pageReq).read()

#         page=urllib.urlopen(req)
        soup=BeautifulSoup(page,'lxml')
        for link in soup.find_all('a'):
            x=link.get('href')
#             print(isinstance(x,str))
            if isinstance(x,str):
            # Parameterization Pending
#                 wordlist=['transcript','2018','q4']
                
                if all(x.find(s)>=0 for s in wordlist):
                    parse_site='http://seekingalpha.com/'+str(x)
                    parse_site='http://seekingalpha.com/'+str(x)+'?part=single'
                    print('Extracting Call Transcripts from: ' + symbol)
                    print(parse_site)
#                     parse_req=urllib.request.urlopen(parse_site)

                    # Open the URL as Browser, not as python urllib
                    pageAct=urllib.request.Request(parse_site,headers=hdr) 
                    infileAct=urllib.request.urlopen(pageAct).read()
            
                    soup = BeautifulSoup(infileAct, 'html5lib') 
                    
                    try:
                        # Get content from article body only
                        text = soup.findAll('div', attrs = {'id':'a-body'})
                        # print(text)

                        for p in text:
                            # Extract all the text from within the p tags
#                             print(p.text)
                            transcriptText += p.text
                        
                    except HTTPError as e:
                            print('Error Code', e.code)
                    transcriptText.replace('\n', ' ')
                    
                    fileName = "../Call_Transcripts_Seeking_Alpha/CallTranscripts" + symbol + "_"+ wordlist[1]+wordlist[2]+ ".txt"
                    with open(fileName, "w") as text_file:
                        print(transcriptText, file=text_file)
                        

    
                    time.sleep(250)
    except HTTPError as e:
        print('Error code:',e.code)

http://seekingalpha.com/symbol/GOOG/earnings/transcripts
Extracting Call Transcripts from: GOOG
http://seekingalpha.com//article/4257884-alphabet-inc-s-goog-ceo-sundar-pichai-q1-2019-results-earnings-call-transcript?part=single
